Task 27.04.20
Bankloan 
Logistic regression
Pipeline
Cross validation

In [106]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
#from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#from sklearn.model_selection import cross_val_score, cross_val_predict
#from category_encoders import WOEEncoder, SumEncoder

In [107]:
os.getcwd()

'D:\\ML\\jupiter_notebook\\Gewissta\\Course_ML\\Модуль_1\\Data'

In [108]:
os.chdir('D:\\ML\\jupiter_notebook\\Gewissta\\Course_ML\\Модуль_1\\Data')

# StateFarm_missing

In [128]:
data = pd.read_csv('StateFarm_missing.csv', sep = ';')

In [129]:
data.head()

,Customer Lifetime Value,Coverage,Education,EmploymentStatus,Gender,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Response
0,2763.519279,Basic,Bachelor,Employed,F,56274.0,NaN,32.0,5.0,NaN,1.0,No
1,NaN,NaN,Bachelor,Unemployed,F,0.0,NaN,13.0,42.0,NaN,NaN,No
2,NaN,NaN,NaN,Employed,F,48767.0,108.0,NaN,38.0,0.0,NaN,No
3,7645.861827,Basic,Bachelor,NaN,NaN,0.0,106.0,18.0,NaN,NaN,7.0,No
4,2813.692575,Basic,Bachelor,NaN,M,43836.0,73.0,12.0,NaN,NaN,1.0,No


In [130]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8293 entries, 0 to 8292
Data columns (total 12 columns):
Customer Lifetime Value          8289 non-null float64
Coverage                         8288 non-null object
Education                        8290 non-null object
EmploymentStatus                 8288 non-null object
Gender                           8289 non-null object
Income                           8291 non-null float64
Monthly Premium Auto             8282 non-null float64
Months Since Last Claim          8288 non-null float64
Months Since Policy Inception    8285 non-null float64
Number of Open Complaints        8287 non-null float64
Number of Policies               8288 non-null float64
Response                         8293 non-null object
dtypes: float64(7), object(5)
memory usage: 777.5+ KB


In [131]:
data.isnull().sum()

Customer Lifetime Value           4
Coverage                          5
Education                         3
EmploymentStatus                  5
Gender                            4
Income                            2
Monthly Premium Auto             11
Months Since Last Claim           5
Months Since Policy Inception     8
Number of Open Complaints         6
Number of Policies                5
Response                          0
dtype: int64

In [132]:
data['Response'].describe()

count     8293
unique       2
top         No
freq      7462
Name: Response, dtype: object

In [133]:
data['Response'] = data['Response'].replace({'No':0, 'Yes':1})

In [134]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Response', axis = 1), data['Response'], 
                                                    test_size = 0.3, stratify = data['Response'],
                                                   random_state = 42)

In [135]:
cat_columns = X_train.dtypes[X_train.dtypes == 'object'].index
num_columns = X_train.dtypes[X_train.dtypes != 'object'].index

In [136]:
num_pipe = Pipeline([('imp', SimpleImputer()), 
                    ('scaler', StandardScaler())])

In [137]:
cat_pipe = Pipeline([('imp', SimpleImputer()), 
                    ('ohe', OneHotEncoder(sparse = False, handle_unknown = 'ignore'))])

In [138]:
transformers = [('num', num_pipe, num_columns), ('cat', cat_pipe, cat_columns)]

In [139]:
transformer = ColumnTransformer(transformers = transformers)

In [140]:
ml_pipe = Pipeline([('tf', transformer),
                    ('logreg', LogisticRegression(solver = 'lbfgs', max_iter = 200))])

In [141]:
param_grid = {
    'tf__num__imp__strategy': ['mean', 'median', 'constant'],    
    'tf__cat__imp__strategy': ['most_frequent', 'constant'],
    'logreg__C': [.01, .1, .5, 1, 5, 10, 100]
}

In [142]:
gs = GridSearchCV(ml_pipe, param_grid, cv = 5, return_train_score = False)

In [143]:
y_train.head()

4065    1
1258    0
2963    0
3339    0
6986    0
Name: Response, dtype: int64

In [149]:
gs.fit(X_train, y_train)
print('best parameters', gs.best_params_)
print('best score', gs.best_score_)
print('Accuracy test', gs.score(X_test, y_test))
#print('All results', gs.cv_results_)

C:\Users\Dell\Anaconda3\envs\py36\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


best parameters {'logreg__C': 0.01, 'tf__cat__imp__strategy': 'most_frequent', 'tf__num__imp__strategy': 'mean'}
best score 0.8997416020671835
Accuracy test 0.89991961414791
Wall time: 8.84 s


In [151]:
results = pd.DataFrame(gs.cv_results_)
table = results.pivot_table(values=['mean_test_score'],    
                            index=['param_logreg__C', 
                                   'param_tf__cat__imp__strategy',
                                   'param_tf__num__imp__strategy'])
print(table)

                                                                           mean_test_score
param_logreg__C param_tf__cat__imp__strategy param_tf__num__imp__strategy                 
0.01            constant                     constant                             0.899742
                                             mean                                 0.899742
                                             median                               0.899742
                most_frequent                constant                             0.899742
                                             mean                                 0.899742
                                             median                               0.899742
0.10            constant                     constant                             0.899569
                                             mean                                 0.899569
                                             median                               0.899569